<a href="https://colab.research.google.com/github/cbonnin88/RailFlow/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import polars as pl
import plotly.express as px

# **Cleaning the Data**

In [ ]:
df_users = pl.read_csv('users.csv', infer_schema_length=0)
print(f'Raw Users Shape: {df_users.shape}')

Raw Users Shape: (5250, 5)


In [3]:
display(df_users['age'].head(10))

age
str
"""34"""
"""90"""
"""81"""
"""84"""
"""81"""
"""81"""
"""41"""
"""47"""
"""55"""


In [4]:
# Cleaning Logic
clean_users = (
    df_users
    .unique(subset=['user_id']) # 1. Remove Duplicate User IDs
    .with_columns([
        # 2. Clean Age: Remove 'yo', 'years', etc. Then cast to Integer
        pl.col('age')
        .str.replace_all(r'[a-zA-Z\s]','') # Regex to remove letter
        .cast(pl.Int64,strict=False)     # Cast to Int (non-parseable becomes null)
        .abs()                        # Turn negative numbers positive
        .alias('age_clean')
    ])
    .filter(
        # 3. Valid Age Filter (18 to 100)
        (pl.col('age_clean') >= 18) & (pl.col('age_clean') <= 100)
    )
    .with_columns(
        # 4. Handle Null Subscription Types (Fill with 'Free)
        pl.col('subscription_type').fill_null('Free')
    )
    .select(['user_id','signup_date','subscription_type','age_clean','device_os'])
    .rename({'age_clean':'age'}) # Rename back
)

In [5]:
print(f'Clean Users Shape: {clean_users.shape}')

Clean Users Shape: (4742, 5)


# **Cleaning Searches Data**

In [6]:
df_searches = pl.read_csv('searches.csv',infer_schema_length=1000)

In [8]:
# Cleaning Logic
clean_searches = (
    df_searches
    .with_columns([
        # 1. Fix Timestamp
        pl.col('timestamp')
        .str.replace_all('/','-') # Standardize separators
        .str.to_datetime(strict=False) # Convert to datetime
    ])
    .drop_nulls(subset=['timestamp']) # Drop rows where timestamp couldn't be parsed
    .with_columns([
        # 2. Convert Departure Date
        pl.col('departure_date').str.to_datetime(strict=False)
    ])
    .with_columns([
        # 3. Standardize Stations: Title Case and Trim Whitespace
        pl.col('origin_station').str.strip_chars().str.to_titlecase(),
        pl.col('destination_station').str.strip_chars().str.to_titlecase()
    ])
    .filter(
        # 4. Logical Filter: Departure must be AFTER Search
        pl.col('departure_date') >= pl.col('timestamp')
    )
)

In [9]:
print(f'Clean Searches Shape: {clean_searches.shape}')

Clean Searches Shape: (15000, 8)


# **Cleaning The Bookings Dataset**

In [10]:
df_bookings = pl.read_csv('bookings.csv')

In [17]:
clean_bookings = (
    df_bookings
    .with_columns([
        # 1. Clean Price: Remove '€' and cast to Float
        pl.col('amount_eur')
        .str.replace('€','')
        .cast(pl.Float64)
    ])
    .filter(
        # 2. Filter Orphan Bookings
        pl.col('search_id').is_in(clean_searches['search_id'].unique().to_list())
    )
)

In [18]:
print(f'Clean booking Shape: {clean_bookings.shape}')

Clean booking Shape: (4000, 5)


In [19]:
# Writing the to a csv
clean_users.write_csv('clean_users.csv')
clean_searches.write_csv('clean_searches.csv')
clean_bookings.write_csv('clean_bookings.csv')

In [20]:
clean_users.head()

user_id,signup_date,subscription_type,age,device_os
str,str,str,i64,str
"""d4f8a8ea-df0b-4f5b-8589-f8f2bd…","""2024-10-18""","""Max Jeune""",36,"""iOS"""
"""e5d121fd-10fa-4cbb-ac2d-548da5…","""2024-05-28""","""Free""",80,"""Unknown"""
"""8c606780-bb5b-4167-bb3e-6c5a87…","""2025-11-08""","""Free""",32,"""iOS"""
"""d9848138-7a75-49e1-b1fd-00f6cb…","""2025-01-14""","""Free""",43,"""Unknown"""
"""b568452d-e88d-4b23-b924-c5ef67…","""2024-12-25""","""Free""",39,"""Unknown"""


In [21]:
clean_searches.head()

search_id,user_id,timestamp,origin_station,destination_station,passenger_count,departure_date,depature_date
str,str,datetime[μs],str,str,i64,datetime[μs],str
"""fba4f473-c45c-44cb-a180-f1f0c5…","""d09d81e5-e17a-416c-b37a-a69be0…",2025-05-10 11:53:09,"""Nantes""","""Strasbourg""",1,2025-05-30 11:53:09.419189,null
"""e979d7ef-1c14-4bd7-b19e-fbf76b…","""bfcade59-80f8-4618-9ccc-de44a7…",2025-01-18 02:45:56.746525,"""Marseille St-Charles""","""Nantes""",1,2025-01-23 02:45:56.746525,null
"""529b14cb-0ed5-4ee0-9228-e4a84d…","""0f2b800f-9f10-433e-9998-06d95e…",2025-05-14 06:00:00.490315,"""Lyon Part-Dieu""","""Nantes""",2,2025-05-21 06:00:00.490315,null
"""1b905016-8e6c-4ea4-b96c-b4b9e8…","""5eab057d-28fa-4bcb-b1ab-a08a2c…",2025-07-03 02:35:23.365211,"""Bordeaux St-Jean""","""Bordeaux St-Jean""",3,2025-08-01 02:35:23.365211,null
"""10bbab91-5dde-45a9-8717-df2cf2…","""64f51332-15dd-467b-9ef1-f4a32b…",2025-10-31 14:05:16.469735,"""Lille Europe""","""Paris Gare De Lyon""",1,2025-11-05 14:05:16.469735,null


In [22]:
clean_bookings.head()

booking_id,search_id,payment_status,ticket_class,amount_eur
str,str,str,str,f64
"""e80dc352-ee1b-4c10-b3eb-ce93bd…","""2fc1ab31-20f9-4577-8bed-544797…","""Success""","""2nd Class""",71.81
"""77b36a1f-fe38-4a8c-9af9-e7cff1…","""de0dbd1d-85f8-436b-b95f-ce4c0f…","""Success""","""2nd Class""",140.79
"""c21dffac-e07b-4338-92df-95cff2…","""2ab924c3-e678-45f6-b838-9248df…","""Success""","""2nd Class""",32.69
"""ccb7e082-734b-4644-9997-7bd3e4…","""f4bb9c14-7546-4df8-87c1-91765e…","""Success""","""2nd Class""",60.48
"""06d439ac-a86b-46ec-aaaf-159bd6…","""eb188485-97a9-4903-9521-b6e5d8…","""Success""","""2nd Class""",132.07
